### chapter7： Data Cleaning and Preparation（数据清洗与数据准备）
pandas和内置的Python标准库提供了一组高级的、 灵活的、快速的工具， 可以让你轻松地将数据规整为想要的格式

    缺失数据、重复数据、字符串操作和其它分析数据转换的工具。

#### 7.1 处理缺失数据

    pandas对象的所有描述性统计默认都不包括缺失数据。对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据

In [1]:
import numpy as np
import pandas as pd

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [4]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

    在pandas中，我们采用了R语言中的惯用法，即将缺失值表示为NA，表示 not available。
    python内置的None值在对象数组中也可以作为NA：

In [6]:
string_data[0] = None

In [7]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

    缺失值处理函数

|方法|说明|
|---|---|
|dropna|根据各标签的值中是否存在缺失数据对轴标签进行过滤，可通过阈值调节对缺失值的容忍度|
|fillna|用指定值或插值方法（如ffill或bfill）填充数据|
|isnull|返回一个含有布尔值的对象，布尔值表示哪些值时缺失值/NA，该对象类型与源类型一样|
|notnull|isnull的否定|

##### 7.1.1 滤除缺失数据

    过滤掉缺失数据的方法：pandas.isnull或布尔型索引的手工方法。但dropna可能会更实用一些。
    对于一个Series，dropna返回一个仅含非空数据索引值的Series。

In [9]:
from numpy import nan as NA

In [10]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [11]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

    以上操作等价于

In [12]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

    对于一个DataFrame对象，根据需求的不同，或许需要丢弃全NA，或需要丢弃含有NA的行或列。
    dropna默认丢弃任何含有缺失值的行：

In [13]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])

In [14]:
cleaned = data.dropna()

In [16]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [17]:
cleaned
# 丢弃了含有缺失值的 1 2 3 行

,0,1,2
0,1.0,6.5,3.0


    传入 how='all' 将只丢弃全为 NA 的那些行

In [18]:
data.dropna(how='all')
# 过滤了全为NA的 2 列

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


    如果用这种方式丢弃列，只需传入axis=1就可以

In [19]:
data[4] = NA
# 新增全为NA的列

In [20]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [23]:
data.dropna(how='all', axis=1)
#  设置了axis参数，值为1，表示是针对列的操作，值为0，表示是对行的操作（但默认没有axis的时候是对行操作，没必要在单独设置axis=0）

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


    另一个滤除DataFrame行的问题涉及时间序列数据。假设只想留下一部分观测数据，可以用thresh参数实现此目的
    thresh = n ,保留 至少有n个非NaN数值的行

In [24]:
df = pd.DataFrame(np.random.randn(7, 3))

In [25]:
df.iloc[:4, 1] = NA
#  0 1 2 3 行 的 1 列 设置为NA

In [35]:
df.iloc[:2, 2] = NA
#  0 1 行 的 2 列 设置为0

In [36]:
df
#  0 1 行有两个NaN，只有一个非NaN的数值

,0,1,2
0,-1.250833,NaN,NaN
1,-1.472979,NaN,NaN
2,-0.561878,NaN,0.881806
3,-0.043848,NaN,0.794250
4,0.425357,0.457719,-0.560822
5,1.215762,0.383718,-1.071181
6,-0.073261,0.208420,-1.121305


In [37]:
df.dropna()
# 所有含有NA的行或列都过滤了

,0,1,2
4,0.425357,0.457719,-0.560822
5,1.215762,0.383718,-1.071181
6,-0.073261,0.208420,-1.121305


In [38]:
df.dropna(thresh=2)
#  设置thresh参数的值为2，即保留非NaN数值数量至少为2的行，0 1 行不满足所以被过滤了

,0,1,2
2,-0.561878,NaN,0.881806
3,-0.043848,NaN,0.794250
4,0.425357,0.457719,-0.560822
5,1.215762,0.383718,-1.071181
6,-0.073261,0.208420,-1.121305


##### 7.1.2 填充缺失数据
    fillna()函数是最主要的填充函数，通过一个常数调用fillna就会将缺失值替换为那个常数值

In [39]:
df.fillna(0)

,0,1,2
0,-1.250833,0.000000,0.000000
1,-1.472979,0.000000,0.000000
2,-0.561878,0.000000,0.881806
3,-0.043848,0.000000,0.794250
4,0.425357,0.457719,-0.560822
5,1.215762,0.383718,-1.071181
6,-0.073261,0.208420,-1.121305


    通过字典调用fillna，可以实现对不同的列填充不同的值

In [42]:
df.fillna({1: 0.5, 2: 1})
# 字典的键需对应DataFrame的列名

,0,1,2
0,-1.250833,0.500000,NaN
1,-1.472979,0.500000,NaN
2,-0.561878,0.500000,0.881806
3,-0.043848,0.500000,0.794250
4,0.425357,0.457719,-0.560822
5,1.215762,0.383718,-1.071181
6,-0.073261,0.208420,-1.121305


    fillna()默认会返回新对象，但也可以对现有对象进行就地修改

In [44]:
_ = df.fillna(0, inplace=True)

In [45]:
df

,0,1,2
0,-1.250833,0.000000,0.000000
1,-1.472979,0.000000,0.000000
2,-0.561878,0.000000,0.881806
3,-0.043848,0.000000,0.794250
4,0.425357,0.457719,-0.560822
5,1.215762,0.383718,-1.071181
6,-0.073261,0.208420,-1.121305


    对reindexing有效的那些插值方法也可用于fillna:

In [47]:
df = pd.DataFrame(np.random.randn(6, 3))

In [48]:
df.iloc[2:, 1] = NA

In [49]:
df.iloc[4:, 2] = NA

In [50]:
df

,0,1,2
0,-0.393319,0.112399,-0.178031
1,-1.188744,0.672258,-0.437591
2,0.783084,NaN,0.024020
3,-0.436820,NaN,0.190791
4,-1.572192,NaN,NaN
5,-0.504741,NaN,NaN


In [51]:
df.fillna(method='ffill')
# method=ffill  用缺失值前面的一个值代替缺失值
# forward fill
# method=bfill / backfill  用缺失值后面的一个值代替缺失值
# method不能和value同时出现
# 默认axis=0，一行一行的，所以是上一行的值来填充

,0,1,2
0,-0.393319,0.112399,-0.178031
1,-1.188744,0.672258,-0.437591
2,0.783084,0.672258,0.024020
3,-0.436820,0.672258,0.190791
4,-1.572192,0.672258,0.190791
5,-0.504741,0.672258,0.190791


In [55]:
df.fillna(method='ffill', limit=2)
# limit 确定填充的个数，这里设置为2 ，只填充2个缺失值
# 默认为axis=0，所以是一行一行往下，填充两个

,0,1,2
0,-0.393319,0.112399,-0.178031
1,-1.188744,0.672258,-0.437591
2,0.783084,0.672258,0.024020
3,-0.436820,0.672258,0.190791
4,-1.572192,NaN,0.190791
5,-0.504741,NaN,0.190791


In [56]:
#  传入 Series的平均值或中位数
data = pd.Series([1., NA, 3.5, NA, 7])

In [58]:
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [57]:
data.fillna(data.mean())
# mean  算术平均数

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

    fillna的参数：
    axis    待填充的轴，默认为axis=0
    inplace 修改调用者对象而不产生副本
    limit   （对于前向和后向填充）可以连续填充的最大数量

#### 7.2 数据转换

##### 7.2.1 移除重复数据

In [3]:
data = pd.DataFrame({
    'k1': ['one', 'two'] * 3 + ['two'],
    'k2': [1, 1, 2, 3, 3, 4, 4]
})

In [4]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


    DataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）

In [5]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

    drop_duplicates()方法，它会返回一个DataFrame，重复的数组会标为False

In [9]:
data.drop_duplicates()
# result = data.drop_duplicates()
# 重复的数据就没有显示了

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


    duplicates()和drop_duplicates()会默认判断全部列，可以指定部分列进行重复项判断。

In [10]:
data['v1'] = range(7)

In [12]:
data['v1']

0    0
1    1
2    2
3    3
4    4
5    5
6    6
Name: v1, dtype: int32

In [11]:
data.drop_duplicates(['k1'])
#  指定判断 k1 列，k1 列全为 one two 去重后只剩两个数据

,k1,k2,v1
0,one,1,0
1,two,1,1


    duplicates和drop_duplicates默认保留的是第一个出现的值的组合。传入keep='last'则保留最后一个

In [16]:
data.drop_duplicates(['k1'], keep='last')
#  传入keep='first' 效果与不传入keep参数一样

,k1,k2,v1
4,one,3,4
6,two,4,6


##### 7.2.2 利用函数或映射进行数据转换

In [28]:
data = pd.DataFrame({
    'food': ['bacon', 'pulled pork', 'bacon',
             'Pastrami', 'corned beef', 'Bacon',
             'pastrami', 'honey ham', 'nova lox'],
    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
})

In [29]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [30]:
# 肉类到动物的映射
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

    Series的map方法可以接受一个函数或含有映射关系的字典型对象

In [31]:
lowercased = data['food'].str.lower()
#  Series的str.lower()方法将首字母大写转为小写

In [32]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [33]:
data['animal'] = lowercased.map(meat_to_animal)
#  调用map方法，

In [34]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [35]:
#  传入一个能够全部完成这些工作的函数：
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

    使用map是一种实现元素级转换以及其他数据清理工作的便捷方法。

##### 7.2.3 替换值

    利用fillna()填充数据时值替换的一种特殊情况。map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方法。

In [38]:
data = pd.Series([1., -999., 2.,-999., -1000., 3.])

In [39]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [40]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [41]:
# 一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：
data.replace([-999, -1000], np.nan)
# 若加上参数 inplace=True ，则该Series源数据也会改变

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [42]:
# 要让每个值有一个不同的替换值，可以传入一个替换列表
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [43]:
# 传入的参数也可以是字典
data.replace({
    -999: np.nan,
    -1000: 0
})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

    data.reaplce与data.str.replace不同，后者做的事字符串的元素级替换

##### 7.2.4 重命名轴索引
    跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同的标签的对象。轴还可以被就地修改，而无需新建一个数据结构。

In [44]:
data = pd.DataFrame(np.arange(12).reshape((3,4)), 
                   index = ['Ohio', 'Colorado', 'New York'],
                   columns = ['one', 'two', 'three', 'four'])

In [45]:
#  跟Series一样，轴索引也有一个map方法：
transform = lambda x: x[:4].upper()
#  该函数只取了设置了前三个字符，并改为大写

In [47]:
data.index

Index(['Ohio', 'Colorado', 'New York'], dtype='object')

In [46]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [48]:
# 赋值给index，对DataFrame进行就地修改
data.index = data.index.map(transform)

In [49]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


    如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename

In [53]:
data.rename(index=str.title, columns=str.upper)
# 用字符串操作规范列

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


    rename可以结合字典型对象实现对部分轴标签的更新：

In [54]:
data.rename(index={
    'OHIO': 'INDIANA',
},
           columns={
               'three': 'peekaboo'
           })

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


    rename可以实现复制DataFrame并对其索引和列标签进行赋值。

In [55]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [56]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


##### 7.2.5 离散化和面元划分
    为了便于分析，连续数据常常被离散化或拆分为“面元”(bin)。pandas.cut函数用来把一组数据分割成离散的区间。
    使用pandas.cut的必须是一维数组，不能是DataFrame。

    pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise') #0.23.4
    参数含义：
    x：被切分的类数组（array-like）数据，必须是1维的（不能用DataFrame）；
    bins：bins是被切割后的区间（或者叫“桶”、“箱”、“面元”），有3中形式：一个int型的标量、标量序列（数组）或者pandas.IntervalIndex 。
        ·一个int型的标量
         当bins为一个int型的标量时，代表将x平分成bins份。x的范围在每侧扩展0.1%，以包括x的最大值和最小值。
        ·标量序列
        标量序列定义了被分割后每一个bin的区间边缘，此时x没有扩展。
        ·pandas.IntervalIndex
        定义要使用的精确区间。
    right：bool型参数，默认为True，表示是否包含区间右部。比如如果bins=[1,2,3]，right=True，则区间为(1,2]，(2,3]；right=False，则区间为(1,2),(2,3)。
    labels：给分割后的bins打标签，比如把年龄x分割成年龄段bins后，可以给年龄段打上诸如青年、中年的标签。labels的长度必须和划分后的区间长度相等，比如bins=[1,2,3]，划分后有2个区间(1,2]，(2,3]，则labels的长度必须为2。如果指定labels=False，则返回x中的数据在第几个bin中（从0开始）。
    retbins：bool型的参数，表示是否将分割后的bins返回，当bins为一个int型的标量时比较有用，这样可以得到划分后的区间，默认为False。
    precision：保留区间小数点的位数，默认为3.
    include_lowest：bool型的参数，表示区间的左边是开还是闭的，默认为false，也就是不包含区间左部（闭）。
    duplicates：是否允许重复区间。有两种选择：raise：不允许，drop：允许。

In [57]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]

In [58]:
#  将这些数据划分为 18到25  26到35  35到60  60以上
bins = [18, 25, 35, 60, 100]

In [59]:
cats = pd.cut(ages, bins)
# pandas 的 cut 函数

In [60]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

    pandas返回的是一个特殊的Categories对象。结果展示了pandas.cut划分的面元。可以将其看成一组表示面元名称的字符串。

In [61]:
cats.codes
# 结果中的数字，描述了数组中对应位置上的数据位于第几个区间中

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [62]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [63]:
pd.value_counts(cats)
#  是pandas.cut结果的面元计数

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [65]:
pd.cut(ages, [18,26,36,61,100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

    传递一个列表或数组到labels，设置自己的面元名称

In [66]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [67]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

    如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数量的最大值和最小值计算等长面元。

In [3]:
data = np.random.rand(20)

In [4]:
data

array([0.16043255, 0.6374882 , 0.71837861, 0.66310625, 0.85831808,
       0.34949809, 0.97182468, 0.16841267, 0.63763274, 0.40713398,
       0.4795964 , 0.95055012, 0.62060514, 0.86744931, 0.97523132,
       0.53597051, 0.12392435, 0.62526771, 0.89465157, 0.3695204 ])

In [5]:
pd.cut(data, 4, precision=2)
#  precision=2,限定小数只有2位
#  4 表示将data分为 4 份

[(0.12, 0.34], (0.55, 0.76], (0.55, 0.76], (0.55, 0.76], (0.76, 0.98], ..., (0.34, 0.55], (0.12, 0.34], (0.55, 0.76], (0.76, 0.98], (0.34, 0.55]]
Length: 20
Categories (4, interval[float64]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.98]]

    qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元。

In [6]:
data = np.random.randn(1000)

In [7]:
cats = pd.qcut(data, 4)

In [8]:
cats

[(0.698, 3.371], (0.698, 3.371], (-0.64, 0.0441], (0.0441, 0.698], (-0.64, 0.0441], ..., (0.0441, 0.698], (-0.64, 0.0441], (-2.798, -0.64], (0.0441, 0.698], (0.0441, 0.698]]
Length: 1000
Categories (4, interval[float64]): [(-2.798, -0.64] < (-0.64, 0.0441] < (0.0441, 0.698] < (0.698, 3.371]]

In [11]:
pd.value_counts(cats)

(0.698, 3.371]     250
(0.0441, 0.698]    250
(-0.64, 0.0441]    250
(-2.798, -0.64]    250
dtype: int64

In [12]:
#  传递自定义的分位数
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.0441, 1.328], (0.0441, 1.328], (-1.35, 0.0441], (0.0441, 1.328], (-1.35, 0.0441], ..., (0.0441, 1.328], (-1.35, 0.0441], (-1.35, 0.0441], (0.0441, 1.328], (0.0441, 1.328]]
Length: 1000
Categories (4, interval[float64]): [(-2.798, -1.35] < (-1.35, 0.0441] < (0.0441, 1.328] < (1.328, 3.371]]

##### 7.2.6 检测和过滤异常值
    过滤和变换异常值（outlier）在很大程度上就是运用数组运算。

In [13]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [15]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.008810,0.038387,-0.013110,-0.010313
std,0.996240,1.032896,0.988753,1.015489
min,-3.046387,-3.357803,-2.844752,-3.057003
25%,-0.703034,-0.683214,-0.686196,-0.687132
50%,0.039471,0.076750,0.004406,0.021067
75%,0.651948,0.763640,0.644820,0.680041
max,2.890440,2.848245,3.296606,3.055042


In [16]:
col = data[2]

In [17]:
# 找出某列中绝对值大小超过3的值：
col[np.abs(col) > 3]

868    3.296606
882    3.113249
Name: 2, dtype: float64

In [29]:
# 要选出全部含有“超过3或-3的值”的行，可以在布尔型DataFrame中使用any方法：
data[(np.abs(data) > 3).any(1)]
# any(1)是把一行中只要有一个数据满足超过-0.1或0.1的值，即把这一行的数据提取出来：

,0,1,2,3
12,-3.046387,-0.495011,-0.548596,0.459297
133,1.449990,-3.357803,1.133057,1.725877
765,-1.382870,1.058807,0.171089,-3.057003
861,0.457022,2.231906,2.102261,3.055042
868,0.521128,0.179476,3.296606,0.244106
882,0.838411,-1.220277,3.113249,-0.686513
935,-0.738770,-3.099334,-1.846582,-0.812886
977,-0.574785,-3.158619,-0.087627,-1.351936


In [30]:
#  将值限制在区间-3到3以内：
data[np.abs(data) > 3] = np.sign(data) * 3
# 根据数据的值是正还是负，np.sign(data)可以生成 1 和 -1 
#  * 3 之后就限制在了 3到-3 之间

In [32]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.008856,0.039003,-0.013519,-0.010311
std,0.996098,1.031002,0.987451,1.015154
min,-3.000000,-3.000000,-2.844752,-3.000000
25%,-0.703034,-0.683214,-0.686196,-0.687132
50%,0.039471,0.076750,0.004406,0.021067
75%,0.651948,0.763640,0.644820,0.680041
max,2.890440,2.848245,3.000000,3.000000


In [33]:
np.sign(data).head()
#  根据数据的值是正还是负，np.sign(data)可以生成1和-1

,0,1,2,3
0,-1.0,1.0,1.0,-1.0
1,-1.0,1.0,-1.0,1.0
2,-1.0,1.0,1.0,-1.0
3,1.0,-1.0,-1.0,1.0
4,1.0,-1.0,1.0,-1.0


##### 7.2.7 排列和随机采样
    利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting， 随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：

In [34]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))

In [38]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [39]:
sampler = np.random.permutation(5)
# numpy.random.permutation(length)用来产生一个随机序列作为索引，再使用这个序列从原来的数据集中按照新的随机顺序产生随机数据集。
# length 为训练数据的个数。

In [40]:
sampler

array([2, 4, 0, 1, 3])

In [42]:
#  基于iloc的索引操作或take函数中使用该数组：
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [43]:
df.take(sampler)
#  sampler  2 4 0 1 3
# take函数，默认axis=0，行随机排列

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15


    如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用sample方法

In [45]:
df.sample(n=3)
#  随机选取、显示了3行数据

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


    要通过替换方式产生样本（允许重复选择），可以传递replace=True到sample:

In [46]:
choices = pd.Series([5, 7, -1, 6, 4])

In [49]:
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [54]:
draws = choices.sample(n=10, replace=True)

In [55]:
draws

0    5
0    5
1    7
3    6
0    5
4    4
1    7
2   -1
0    5
3    6
dtype: int64

##### 7.2.8 计算指标/哑变量
    另一种常用于统计建模或机器学习的转换方法是：将分类变量（categorical variable）转换为“哑变量”或“指示矩阵”：

In [57]:
#  DataFrame的某一列含有K个不同的值，则可以派生出一个K列矩阵或DataFrame（其值全为1和0）
df = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'b'],
    'datal': range(6)
})

In [60]:
df

,key,datal
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [59]:
#  pandas的get_dummies函数可以实现该功能。
pd.get_dummies(df['key'])
# 按照 key 列，有 a b c 三个不同的值，派生出一个3列的矩阵
# 对应的：b 在0  1 5 列出现，所以下面的矩阵中 b 列的 0 1 5 行值为1

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [61]:
# 给DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。
# get_dummies的prefix参数可以实现该功能：
dummies = pd.get_dummies(df['key'], prefix='key')
# 设置了prefix参数，列名由原来的 a b c  变为了 key_a, key_b, key_c

In [64]:
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [62]:
df_with_dummy = df[['datal']].join(dummies)
# join，连接 datal 列 与 dummies

In [63]:
df_with_dummy

,datal,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [66]:
# 如果DataFrame中的某行同属于多个分类，则会相对复杂
# MovieLens 1M 数据集：
mnames = ['movie_id', 'title', 'genres']

In [73]:
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [74]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [75]:
#  要为每个genre添加指标变量就需要做一些数据规整操作。
# 首先，从数据集中抽出不同的genre值：
all_genres = []

In [76]:
for x in movies.genres:
    all_genres.extend(x.split('|'))
# 先将每一行的genres列的值按'|'进行分割，形成列表，添加到定义好的列表中

In [77]:
genres = pd.unique(all_genres)
#  unique函数，进行去重，使每一个值只存在一个

In [78]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [79]:
# 构建指标DataFrame的方法之一是从一个全零DataFrame开始：
zero_matrix = np.zeros((len(movies), len(genres)))

In [80]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [82]:
dummies.head()
# 用zeros函数创建的，值全为0

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
# 迭代每一步电影，并将dummies各行的条目设为1。
# 使用dummies.columns来计算每个类型的列索引：
gen = movies.genres[0]

In [86]:
gen

"Animation|Children's|Comedy"

In [85]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [88]:
# dummies.columns计算每个类型的列索引
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [90]:
#  根据索引，使用iloc设定值：
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [92]:
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
#  然后，将其余movies合并起来
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [94]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

    对于很大的数据，用这中方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入numpy数组，然后结果包装在DataFrame中。
    一个对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数

In [2]:
# 使用seed，使这个列子具有确定性。
np.random.seed(12345)
# 随机数种子对后面的结果一直有影响。同时，加了随机数种子以后，后面的随机数组都是按一定的顺序生成的。
# 随机数种子的参数怎么选择？我认为随意，这个参数只是确定一下随机数的起始位置。

In [3]:
values = np.random.rand(10)

In [4]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [5]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [6]:
pd.get_dummies(pd.cut(values, bins))
# get_dummies 派生出矩阵

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

#### 7.3 字符串操作
    pandas能够对整组数据应用字符串表达式和正则表达式，而且能够处理缺失数据。

##### 7.3.1 字符串对象方法

    以逗号分隔的字符串可以用split拆分成数端：

In [7]:
val = 'a,b, guido'

In [8]:
val.split(',')

['a', 'b', ' guido']

    split常常与strip一起使用，以出去空白符（包括换行符）：

In [9]:
pieces = [x.strip() for x in val.split(',')]

In [10]:
pieces

['a', 'b', 'guido']

    利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来：

In [11]:
first, second, third = pieces

In [12]:
first + '::' + second + '::' + third

'a::b::guido'

In [13]:
# 另一种实现方法：
'::'.join(pieces)

'a::b::guido'

    检测子串的最佳方法是：python的in关键字，还可以使用index和find：

In [14]:
'guido' in val

True

In [15]:
val.index(',')

1

In [16]:
val.find(':')
# 返回的是-1，表示没有这个字串

-1

In [17]:
val.index(':')
# index方法：找不到该字串则会返回一个异常，而不是返回-1.

ValueError: substring not found

    count可以返回指定子串的出现次数：

In [18]:
val.count(',')

2

    replace用于将指定模式替换为另一模式。传入空字符串时，也可以用于删除模式：

In [19]:
val.replace(',', '::')

'a::b:: guido'

In [20]:
val.replace(',', '')

'ab guido'

##### 7.3.2 正则表达式

    python内置的re模块负责对字符串应用正则表达式，该模块的函数可以分为三个大类：模式匹配、替换、拆分。

In [2]:
# 描述一个或多个空白符的Regex是 '\s+'
import re

In [3]:
text = "foo    bar\t baz   \tqux"

In [4]:
# 调用re.split('\s', text)时，正则表达式会先被编译，然后再text行调用其split方法。
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [5]:
# re.compile 得到可重用的Regex对象
regex = re.compile('\s+')

In [6]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

    如果只希望得到匹配Regex的所有模式，则可以使用findall方法：

In [7]:
regex.findall(text)

['    ', '\t ', '   \t']

    match和search跟findall功能类似。
    findall：返回字符串中所有的匹配项。
    search：返回第一个匹配项。
    match：只匹配字符串的首部。

In [12]:
text = """Dave dave@goole.com
    Steve steve@gmail.com
    Rob rob@gmail.com
    Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [15]:
# 对text使用findall将得到一组电子邮件地址
regex.findall(text)

['dave@goole.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [13]:
# search返回文本中的第一个地址。
m = regex.search(text)
# 这个regex，匹配项对象只能告诉我们模式在原字符串中的起始和结束位置

In [14]:
m
# 起始位置 5 ，结束位置 19

<re.Match object; span=(5, 19), match='dave@goole.com'>

In [18]:
m.start()

5

In [17]:
text[m.start():m.end()]

'dave@goole.com'

In [19]:
#  regex.match 将返回None，因为它只匹配出现在字符串开头的模式：
print(regex.match(text))

None


    相关的，sub方法可以将匹配到的模式替换为指定字符串，并返回所得到的的新字符串：

In [20]:
print(regex.sub('REDACTED', text))

Dave REDACTED
    Steve REDACTED
    Rob REDACTED
    Ryan REDACTED



In [21]:
# 对电子邮件地址进行拆分：用户名、域名以及域后缀
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
# 加了括号，进行了分组----group----

In [22]:
regex = re.compile(pattern, flags=re.IGNORECASE)

    可以通过其groups方法返回一个由模式隔断组成的元祖

In [23]:
m = regex.match('wesm@bright.net')

In [24]:
m.groups()

('wesm', 'bright', 'net')

    对于带有分组功能的模式，findall会返回一个元祖列表

In [25]:
regex.findall(text)

[('dave', 'goole', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

    sub还能通过诸如\1,\2之类的特殊符号访问各匹配项中的分组。符号\1对应第一个匹配的组，\2对应第二个匹配的组。

In [27]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: goole, Suffix: com
    Steve Username: steve, Domain: gmail, Suffix: com
    Rob Username: rob, Domain: gmail, Suffix: com
    Ryan Username: ryan, Domain: yahoo, Suffix: com



##### 7.3.3 pandas的矢量化字符串函数

In [33]:
data = {
    'Dave': 'deve@google.com',
    'Steve': 'steve@gmail.com',
    'Rob': 'rob@gmail.com',
    'Wes': np.nan
}

In [34]:
data = pd.Series(data)

In [35]:
data

Dave     deve@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [36]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

    通过data.map，所有字符串和正则表达式方法都能被以用于（传入lambda表达式或其他函数）各个值，但是如果存在NA，就会报错。为了解决这个问题，Series有一些能够跳过NA值的面向数组方法，进行字符串操作。通过Series的str属性即可访问这些方法。
    例如：可以通过str.contains检查各个电子邮件地址是否含有'gmail':

In [37]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [38]:
# 也可以使用正则表达式，还可以加上任意re选项（如：IGNORECASE）:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [39]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(deve, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

    实现矢量化的元素获取操作：
    str.get;
    在str属性上使用索引

In [40]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

In [41]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [46]:
# 要访问嵌入列表中的元素，可以传递索引到这两个函数中
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [47]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [48]:
# 对字符串进行截取
data.str[:5]

Dave     deve@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object